# **Αναγνώριση Προτύπων – Μηχανική Μάθηση Εργασία 3: Support Vector Machines**

### Κυριακόπουλος Βασίλειος 1115201700064

**Εισαγωγή απαραίτητων βιβλιοθηκών:**

In [1]:
import numpy as np
from keras.datasets import mnist
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA

**Φόρτωση MNIST handwritten digits dataset:**

In [2]:
# Load the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Reshape 28 * 28 images to 784 dimensional vectors
X_train = X_train.reshape((-1,784)).astype(np.float64)
X_test = X_test.reshape((-1,784)).astype(np.float64)
# Normalize the data to [0,1] range
X_train/=X_train.max()
X_test/=X_test.max()

11501568/11490434 [==============================] - 0s 0us/step


**Πειραματισμός με τα SVM στα δεδομένα αρχικής διάστασης:**

Θα δοκιμάσω διάφορα kernels και τις αντίστοιχες παραμέτρους τους και αυτό που θα δώσει την καλύτερη απόδοση, θα το χρησιμποιήσω στο επόμενο ζήτημα με την PCA:

Linear kernel:

In [18]:
%%time
linsvm = LinearSVC(dual=False,C=1.5)
linsvm.fit(X_train,y_train)
print("Linear kernel original dimension accuracy:",linsvm.score(X_test,y_test))

Linear kernel original dimension accuracy: 0.9178
CPU times: user 39.2 s, sys: 62 ms, total: 39.3 s
Wall time: 39 s


RBF kernel:

In [5]:
%%time
rbfsvm = SVC(kernel='rbf',C=2.5,gamma='scale')
rbfsvm.fit(X_train,y_train)
print("RBF kernel original dimension accuracy:",rbfsvm.score(X_test,y_test))

RBF kernel original dimension accuracy: 0.9835
CPU times: user 8min 10s, sys: 487 ms, total: 8min 10s
Wall time: 8min 8s


Poly kernel:

In [17]:
%%time
polysvm = SVC(kernel='poly',C=2.0)
polysvm.fit(X_train,y_train)
print("Poly kernel original dimension accuracy:",polysvm.score(X_test,y_test))

Poly kernel original dimension accuracy: 0.9785
CPU times: user 7min 43s, sys: 481 ms, total: 7min 43s
Wall time: 7min 41s


Μετά από διάφορα πειράματα κατέληξα σε rbf kernel με gamma='scale' 
(1 / (n_features * X.var())) και C=2.5

**Εφαρμογή PCA:**

Θα ορίσω μία συνάρτηση η οποία θα δέχεται σαν παράμετρο το επιθυμητό ποσοστό διατηρούμενης διακύμανσης και θα εκτελεί το αντίστοιχο πείραμα ταξινόμησης με svm (χρησιμοποιώντας τις παραμέτρους που μου έδωσαν το βέλτιστο μοντέλο στα δεδομένα της αρχικής διάστασης) στα δεδομένα μειωμένης διάστασης:

In [11]:
def svm_with_pca(explained_variance_ratio=0.85):
  pca = PCA(n_components=explained_variance_ratio)
  X_train_reduced = pca.fit_transform(X_train)
  X_test_reduced = pca.transform(X_test)

  rbfsvm = SVC(kernel='rbf',C=2.5,gamma='scale')
  rbfsvm.fit(X_train_reduced,y_train)
  print("Number of components used:",pca.n_components_)
  print("RBF kernel reduced dimension accuracy:",rbfsvm.score(X_test_reduced,y_test))

Έπειτα, ας εκτελέσουμε 3 πειράματα:


85% διατηρούμενη διακύμανση:

In [12]:
%%time
svm_with_pca(explained_variance_ratio=0.85)

Number of components used: 59
RBF kernel reduced dimension accuracy: 0.9859
CPU times: user 1min 7s, sys: 1.33 s, total: 1min 8s
Wall time: 57.7 s


88% διατηρούμενη διακύμανση:

In [13]:
%%time
svm_with_pca(explained_variance_ratio=0.88)

Number of components used: 74
RBF kernel reduced dimension accuracy: 0.9859
CPU times: user 1min 17s, sys: 1.09 s, total: 1min 18s
Wall time: 1min 7s


90% διατηρούμενη διακύμανση:

In [14]:
%%time
svm_with_pca(explained_variance_ratio=0.90)

Number of components used: 87
RBF kernel reduced dimension accuracy: 0.9853
CPU times: user 1min 26s, sys: 1.24 s, total: 1min 28s
Wall time: 1min 17s


Παρατηρούμε ότι αρχικά με την μείωση των διαστάσεων μειώνονται οι χρόνοι εκτέλεσης των πειραμάτων κατα περίπου 10 φορές το οποίο είναι φυσιολογικό αφού επεξεργαζόμαστε δεδομένα με περίπου υποδεκαπλάσιο πλήθος διαστάσεων. Τώρα όσον αφορά την απόδοση της ταξινόμησης στα δεδομένα μειωμένης διάστασης, εφόσον στα αντίστοιχα πειράμτα επιλέχθηκε το πλήθος της διατηρούμενης διακύμανσης να είναι μεταξύ του θεωρητικού βέλτιστου 85-90%, παρατηρούμε ότι παίρνουμε ακόμα και μεγαλύτερη ακρίβεια ταξινόμησης από τα πειράματα που έγιναν στα δεδομένα αρχικής διάστασης. Αυτό οφείλεται σε 2 παράγοντες:


1.   Στο γεγονός ότι επιλέγοντας το πλήθος διατηρούμενης διακύμανσης να είναι   μεταξύ 85% και 90% διατηρείται η περισσότερη χρήσιμη πληροφορία στα δεδομένα μειωμένης διάστασης.
2.   Στο γεγονός ότι οι αποστάσεις στους χώρους μειωμένης διάστασης είναι μικρότερες το οποίο συμνεπάγεται ότι μπορούμε να ξεχωρίσουμε τα δεδομένα πιο εύκολα μεταξύ τους. Με άλλα λόγια, ο χώρος μειωμένης διάστασης είναι πιο απλός και δεν ισχύει η κατάρα της διαστατικότητας(curse of dimensionality).

Επιπλέον, όπως είναι αναμενόμενο, όσο αυξάνεται το πλήθος των κύριων συνιστωσών τόσο περισσότερο αυξάνεται και ο χρόνος εκτέλεσης του αντίστοιχου πειράματος και αρχίζει να μειώνεται σιγα σιγά και η ακρίβεια ταξινόμησης λόγω όσων ειπώθηκαν παραπάνω στο 2.

